In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('AB_NYC_2019.csv')

In [3]:
X = data.drop(['id', 'name','host_name', 'last_review', 'reviews_per_month'], axis = 1)
X.dropna(axis = 0, inplace = True)
y = X['price']
X.drop(['price'], axis = 1, inplace = True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)

In [5]:
X_train.shape

(39116, 10)

In [6]:
y_test.shape

(9779,)

In [7]:
X_test.shape

(9779, 10)

In [8]:
from sklearn.preprocessing import LabelEncoder

cols = ['neighbourhood_group', 'neighbourhood', 'room_type']
for col in cols :
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

<ipython-input-8-1c282ddd56ee>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = le.fit_transform(X_train[col])
<ipython-input-8-1c282ddd56ee>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = le.transform(X_test[col])


In [9]:
X_train.head()

,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
47458,265866685,1,28,40.69796,-73.92915,0,30,0,2,179
19782,99602138,2,206,40.83490,-73.94829,0,3,36,1,5
31472,75173477,1,28,40.69932,-73.91191,0,3,7,1,6
20323,36382944,1,214,40.71520,-73.96230,0,31,2,1,0
32294,22935245,1,13,40.69168,-73.95397,0,21,1,1,25


In [10]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

In [12]:
print(tf. __version__)

2.6.0


In [13]:
import tensorflow.compat.v1 as tf
tf.get_default_graph()

In [22]:
def embedding_model():
    model = Sequential()
    model.add(Embedding(input_dim = 10, output_dim = 5))
    model.add(Conv1D(5, 1, activation = 'relu'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(5, activation = 'relu'))
    model.add(Dense(1, activation = 'linear'))
    return model

In [23]:
from tensorflow.keras.optimizers import Adam

In [24]:
from sklearn.model_selection import StratifiedKFold
import math
import numpy as np
from sklearn.metrics import mean_squared_error

oof = np.zeros((len(X_train), 1))
preds = np.zeros((len(X_test), 1))

skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2020)

for fold, (trn_idx, test_idx) in enumerate(skf.split(X_train, y_train)):
    trn_X, test_X = X_train[trn_idx], X_train[test_idx]
    trn_y, test_y = y_train.iloc[trn_idx], y_train.iloc[test_idx]
    
    model = embedding_model()
    model.compile(loss = 'mse', metrics = [tf.keras.metrics.RootMeanSquaredError()], optimizer = 'adam')
    model.fit(trn_X, trn_y, batch_size = 128, epochs = 100, validation_data = (test_X, test_y), verbose = 0)
    preds = model.predict(test_X)
    oof[test_idx] += preds
    score = np.sqrt(mean_squared_error(test_y, preds))
    preds += model.predict(X_test) / 5

C:\Users\pooja\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


ValueError: The last dimension of the inputs to `Dense` should be defined. Found `None`.

In [ ]:
print('Train RMSE : ', np.sqrt(mean_squared_error(y_train, oof)))
print('Test RMSE :', np.sqrt(mean_sqaured_error(y_test, preds)))